In [1]:
import numpy as np
import torch

# PyTorchテンソルの出力表示を見やすく設定
torch.set_printoptions(
    edgeitems=2,  # 各次元の先頭・末尾2個の要素だけ表示（中間は "..." で省略）
    threshold=50,  # 要素数がこれを超えると省略表示になる
    linewidth=75,  # 1行の最大表示幅。これを超えると自動改行される
)

In [ ]:
# CSVファイルからバイクシェアデータ（時間単位）を読み込み
bikes_numpy = np.loadtxt(
    "../../data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
    dtype=np.float32,  # 全データを float32 型として読み込む
    delimiter=",",  # カンマ区切りのCSVファイル
    skiprows=1,  # ヘッダー行をスキップ
    converters={
        1: lambda x: float(x[8:10])  # <1> 2列目（index=1）を "日付から日だけ抽出"
        # 例: "2011-01-01" → x[8:10] → "01" → float → 1.0
    },
)

# NumPy配列からPyTorchテンソルへ変換
bikes = torch.from_numpy(bikes_numpy)

# 結果を表示（テンソルの形状や内容の確認）
bikes

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

In [5]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [6]:
# bikes: shape = [17520, 17]（17520時間分、各時点17特徴量）

# 1日24時間ごとに区切って reshape（view）→ [日数, 24時間, 特徴量]
daily_bikes = bikes.view(-1, 24, bikes.shape[1])

# 結果の形状とストライドを確認
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [7]:
# daily_bikes: shape = (730, 17, 24)
# これは [日数, 特徴量数, 時間数] の構造で、各日について各特徴量の時間変化を表している

# transpose(1, 2) により、特徴量軸（1）と時間軸（2）を入れ替える
# 結果：shape = (730, 24, 17) となり、
# 各日について「時間ごとの特徴量ベクトル」が並ぶ構造に変わる
# これは例えば RNN で時間ごとの入力を処理したい場合などに便利な形
daily_bikes = daily_bikes.transpose(1, 2)

daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))